In [22]:
import os
from pathlib import Path
import glob
import json
import random
import shutil
from utils import wavwrite, read_scaled_wav, fix_length
from random import randrange

eat_dir = "D:/DP_files/EAT-2mix/min/"

tai_dir = "D:/DP_files/condenser/"
eng_dir = "D:/DP_files/wsj0/"
noise_dir = "D:/DP_files/wham_noise/"

# Create output directories
for mode in ["tr", "cv", "tt"]:
    os.makedirs(eat_dir + mode + "/" + mix_clean_dir)
    os.makedirs(eat_dir + mode + "/" + mix_both_dir)
    os.makedirs(eat_dir + mode + "/" + speaker1_dir)
    os.makedirs(eat_dir + mode + "/" + speaker2_dir)

# Collect filenames from input directories
tai_list = [w for w in Path(tai_dir).glob('**/*.wav') if os.path.isfile(w)]
tai_list.sort()
eng_list = [w for w in Path(eng_dir).glob('**/*.wav') if os.path.isfile(w)]
eng_list.sort()
noise_list = [w for w in Path(noise_dir).glob('**/*.wav') if os.path.isfile(w)]
noise_list.sort()

# Combine speakers and save files to output directory
tai_counter = 0
for i in range(28000):
        # Load audio data from input files
        audio1 = read_scaled_wav(tai_list[tai_counter], 1, 8000)
        audio2 = read_scaled_wav(eng_list[i], 1, 8000)
        noise = read_scaled_wav(noise_list[i], 1, 8000)

        # Mix audio and save to output file
        audio1, audio2 = fix_length(audio1, audio2, "tat") 
        # přepsal jsem si fix_length "eat" ať to bere konec nahrávky, protože v tat mluví nakonci  a na začátku je ticho
        if(len(noise) > len(audio1)):
            audio1, noise = fix_length(audio1, noise, "min")
        else:
            audio1, noise = fix_length(audio1, noise, "max")

        mixed_speech = audio1 + audio2
        mixed_noisy = audio1 + audio2 + noise
        if i < 20000:
            output_file = f"{i}.wav"
            wavwrite(eat_dir + "tr/mix_clean/" + output_file, mixed_speech, 8000)
            wavwrite(eat_dir + "tr/mix_both/" + output_file, mixed_noisy, 8000)
            wavwrite(eat_dir + "tr/s1/" + output_file, audio1, 8000)
            wavwrite(eat_dir + "tr/s2/" + output_file, audio2, 8000)
        elif i < 25000:
            output_file = f"{i-19999}.wav"
            wavwrite(eat_dir + "cv/mix_clean/" + output_file, mixed_speech, 8000)
            wavwrite(eat_dir + "cv/mix_both/" + output_file, mixed_noisy, 8000)
            wavwrite(eat_dir + "cv/s1/" + output_file, audio1, 8000)
            wavwrite(eat_dir + "cv/s2/" + output_file, audio2, 8000)
        elif i < 28000:
            output_file = f"{i-24999}.wav"
            wavwrite(eat_dir + "tt/mix_clean/" + output_file, mixed_speech, 8000)
            wavwrite(eat_dir + "tt/mix_both/" + output_file, mixed_noisy, 8000)
            wavwrite(eat_dir + "tt/s1/" + output_file, audio1, 8000)
            wavwrite(eat_dir + "tt/s2/" + output_file, audio2, 8000)

        tai_counter += 1
        if tai_counter == len(tai_list):
            tai_counter = 0